In [32]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import glob 

In [14]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset = dataset.repeat(3)
dataset = dataset.map(lambda x: x * 2)
dataset = dataset.shuffle(buffer_size = 5).batch(7, drop_remainder = True)
for item in dataset:
    print(item)

tf.Tensor([ 6  0 10  2  4  8 12], shape=(7,), dtype=int32)
tf.Tensor([16  0  4  6 10 14  2], shape=(7,), dtype=int32)
tf.Tensor([18 18 12  8  0 14  4], shape=(7,), dtype=int32)
tf.Tensor([ 8  6 12 10 16 18  2], shape=(7,), dtype=int32)


In [22]:
housing_data = pd.read_csv('./datasets/housing/housing.csv')

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
96,-122.26,37.82,31.0,4596.0,1331.0,2048.0,1180.0,2.8345,183800.0,NEAR BAY
97,-122.26,37.81,29.0,335.0,107.0,202.0,91.0,2.0062,125000.0,NEAR BAY
98,-122.26,37.82,22.0,3682.0,1270.0,2024.0,1250.0,1.2185,170000.0,NEAR BAY
99,-122.26,37.82,37.0,3633.0,1085.0,1838.0,980.0,2.6104,193100.0,NEAR BAY


In [30]:
train_data = housing_data.loc[:15000]
val_data = housing_data.loc[15000:18000].reset_index(drop=True)
test_data = housing_data.loc[18000:].reset_index(drop=True)
ctr = 0
for i in range(1000,15000,1000):
    train_data.loc[ctr:i].to_csv('./datasets/housing/train{0}.csv'.format(ctr))
    ctr = i
ctr = 0
for i in range(1000,3000,1000):
    val_data.loc[ctr:i].to_csv('./datasets/housing/val{0}.csv'.format(ctr))
    ctr = i
ctr = 0
for i in range(1000,len(test_data),1000):
    test_data.loc[ctr:i].to_csv('./datasets/housing/test{0}.csv'.format(ctr))
    ctr = i

In [37]:
train_filepaths = glob.glob('./datasets/housing/train*.csv')
val_filepaths = glob.glob('./datasets/housing/val*.csv')
test_filepaths = glob.glob('./datasets/housing/test*.csv')
filepath_dataset = tf.data.Dataset.list_files(train_filepaths,seed=42)

<ShuffleDataset shapes: (), types: tf.string>

In [39]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

In [58]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_data)
X_mean = scaler.mean_
X_std = scaler.scale_

ModuleNotFoundError: No module named 'sklearn'

In [56]:
n_inputs = 8
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype = tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    print(x)
    return (x-X_mean)/X_std,y

preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')


tf.Tensor(
[   4.2083   44.        5.3232    0.9171  846.        2.337    37.47
 -122.2   ], shape=(8,), dtype=float32)


InvalidArgumentError: Incompatible shapes: [8] vs. [9] [Op:Sub]